# Age Prediction Sandbox Notebook

In [ ]:
import numpy as np
import SimpleITK as sitk
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
# config
version = '50'
project_root = os.path.abspath('..')
load_dir = os.path.join(project_root, 'data/input/unsegmented')
segmentations_dir = os.path.join(project_root, f'data/output/segmentations/{version}')


In [ ]:
# Load segmentations and labels
labels = []
images = []
for index, file_name in enumerate(sorted(glob.glob(os.path.join(load_dir, '*.nii.gz')))):
    segmentaiton_path = os.path.join(segmentations_dir, f'{index}.nii.gz')
    img = sitk.ReadImage(segmentaiton_path)
    images.append(sitk.GetArrayFromImage(img))
    
    age_str = file_name.split('_')[-1].split('.')[:-2]
    age = int(age_str[0]) + (len(age_str)-1)*int(age_str[-1])/10.
    labels.append(age)
    #print(images[-1].shape, age_str)

In [ ]:
plt.imshow(images[0][99], cmap='gray')
plt.axis('off')
plt.title('An example of generated segmentation.')
plt.show()

In [ ]:
n_samples = len(images)
n_features = 3
features = np.zeros((n_samples, n_features))

In [ ]:
# Create numpy featere set and labels
for i, img in enumerate(images):
    features[i, 0] = (img == 1).sum()
    features[i, 1] = (img == 2).sum()
    features[i, 2] = (img == 3).sum()
    
features = features / features.sum(axis=-1, keepdims=True)
labels = np.array(labels).reshape(-1,1)

In [ ]:
# Split the dataset into train and test part
train_val_features, test_features, train_val_labels, test_labels = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# Define parameters for grid search
parameters = {'poly__degree':[2,3,4,5,6,7,8, 9, 10,11,12], 'ridge__alpha': np.logspace(-6, 6, 13),}

In [ ]:
# Define the pipeline of the model
model = Pipeline(steps = [('poly', PolynomialFeatures()),
                        ('ridge', Ridge(fit_intercept=False))])

#model = Pipeline(steps = [('poly', PolynomialFeatures()),
#                        ('lasso', Lasso(fit_intercept=False))])

#model = Pipeline(steps = [('poly', PolynomialFeatures()),
#                        ('e-net', ElasticNet(fit_intercept=False))])

In [ ]:
# Perform gridsearch using 5-fold crossvalidation
clf = GridSearchCV(model, parameters, )
clf.fit(train_val_features, train_val_labels[:, 0])
print('Done')

In [ ]:
print('Best results according to the grid search:')
print(clf.best_params_)

In [ ]:
# Apply the best model to training part
pred = clf.predict(train_val_features)
mae = np.mean(np.abs(pred - train_val_labels[:,0]))
std = np.std(np.abs(pred - train_val_labels[:,0]))

In [ ]:
print(f'Prediction:{pred}\nLabels: {train_val_labels[:,0]}')

In [ ]:
print(f'MAE: {mae}, std: {std}')

In [ ]:
# Apply the best model to training part
pred_test = clf.predict(test_features)
mae_test = np.mean(np.abs(pred_test - test_labels[:,0]))
std_test = np.std(np.abs(pred_test - test_labels[:,0]))

In [ ]:
print(f'MAE: {mae_test}, std: {std_test}')

Let's train a neural network with couple of linear layes for fun

In [ ]:
import torch

In [ ]:
# Define Model Structure

dropout_prob = .2
in_channels = 3
mid_channels = 8
out_channels = 1

model = torch.nn.Sequential(
    torch.nn.Linear(in_channels, mid_channels),
    torch.nn.ReLU(),
    torch.nn.Dropout(dropout_prob),
    torch.nn.Linear(mid_channels, 2 * mid_channels),
    torch.nn.ReLU(),
    torch.nn.Dropout(dropout_prob),
    torch.nn.Linear(2 * mid_channels, mid_channels),
    torch.nn.ReLU(),
    torch.nn.Dropout(dropout_prob),
    torch.nn.Linear(mid_channels, out_channels)
)

# define your optimizer and loss function
optimiser = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

# convert your training arrays to tensors of the correct type

X = torch.from_numpy(train_val_features).float()
y = torch.from_numpy(train_val_labels).float()

num_epochs = 4000  # number of runs through the dataset to train for
# batch size equals to the size of dataset



In [ ]:
# training loop
losses = []
for epoch in range(1, 1 + num_epochs):
    model.train()
    optimiser.zero_grad()
    pred = model(X)
    loss = criterion(pred, y)
    losses.append(loss.item())
    loss.backward()
    optimiser.step()

In [ ]:
# Convert the test data to tensors and test your network against the test ground truth data

X_test = torch.from_numpy(test_features).float()
y_test = torch.from_numpy(test_labels).float()

model.train() # To keep Dropout active and produce output for ensemble

pred = np.zeros((4,1))
# Produce ensemble output
for i in range(10):
    pred += model(X_test).detach().numpy() 
pred /= 10

In [ ]:
print('Patient | Prediction | Target')
for i in range(len(y_test)):
    print(f'  {i}     |   {pred.squeeze()[i]:.2f}    | {y_test.squeeze()[i]:.2f}')


In [ ]:
plt.plot(losses)
plt.title('MSE convergence')
plt.xlabel('Iteration')
plt.grid()
plt.show()